In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [2]:
path = os.getcwd() + "/atribuna/milarquivos"
content_file_array = []
classifier_array = []

In [3]:
def read_text(path, file_Name):
    with open(path, encoding="latin-1") as file:
        content_file = file.read()
        Build_DataFrame(content_file, file_Name)

In [4]:
def Build_DataFrame(content_file, file_Name):
    classifier =  GetClassifier(file_Name) 
    if(len(classifier) >= 3):        
        classifier_array.append(classifier)
        content_file_array.append(content_file)  

In [5]:
def GetClassifier(file_Name):
    name_split =  file_Name.split(".")
    first_name = name_split[0];
    return first_name[(len(first_name) - 3) : len(first_name)]

In [6]:
for file_Name in tqdm(os.listdir(path), desc="Carregar arquivos"):    
    if file_Name.endswith(".txt"):
        file_path = f"{path}\{file_Name}"
      
        # call read text file function
        read_text(file_path, file_Name)

Carregar arquivos: 100%|█████████████████████████████████████████████████████████████| 69/69 [00:00<00:00, 5308.66it/s]


In [7]:
textDataDrame =  pd.DataFrame({"Text":content_file_array, "Class":classifier_array})


In [8]:
textDataDrame.head(10)

,Text,Class
0,Versatilidade é com ela\n<b> No CD Nos Horizo...,at2
1,"Vale até um empurrão do pai\n<b> Às vezes, mãe...",esp
2,Idosos pressionados pela família\n<b> Eles são...,fam
3,Ela só pensa em fazer maldades\n<b> Adriana Ga...,tvt
4,Prefeituras definem projetos para cidades\n<b>...,cid
5,PLANETA POP\n<b> </b><br>\n <br>\nAngelina Jo...,at2
6,Na melhor idade do rock\n<b> Músicos que estão...,at2
7,Dez leitores faturam TVs de tela plana\n<b> Os...,cid
8,PLANETA POP\n<b> </b><br>\n <br>\nLindsay Loh...,at2
9,PLANETA POP\n<b> </b><br>\n <br>\n24 Horas ...,at2


# StopWords removing

In [9]:
#!conda install unidecode

import nltk
import unidecode
from bs4 import BeautifulSoup

# nltk.download()
#- averaged_perceptron_tagger
# - floresta
# - mac_morpho
# - machado
# - punkt
# - stopwords
# - wordnet
# - words

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [10]:
def addStopWordsToList(texto):
    example_sent = texto
    word_tokens = word_tokenize(example_sent)

    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

    filtered_sentence = ""

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence += w + " "
    return filtered_sentence.strip()

In [11]:
stop_words = set(stopwords.words('portuguese'))
def returnDataFrameStopWords(dataFrame):
    dataReturn = pd.DataFrame()
    for row in tqdm(dataFrame.itertuples(index=True, name='Pandas'), desc="Limpeza de texto"):
        Text = row.Text
        Class = row.Class

        Text = BeautifulSoup(Text, "lxml").text
        Text = unidecode.unidecode(Text)
        Text = addStopWordsToList(Text)
        Text = Text.replace(" . "," ")
        Text = Text.replace(" , "," ")
        Text = Text.replace(" - "," ")
        Text = Text.replace(" : "," ")
        Text = Text.lower()

        linha = pd.Series([Text, Class])
        linha = pd.DataFrame([linha])
        dataReturn = pd.concat([linha, dataReturn], ignore_index=True)

    return dataReturn

In [12]:
df_text_clean = returnDataFrameStopWords(textDataDrame)

Limpeza de texto: 69it [00:01, 63.65it/s]


## Fatores de Ponderação

In [13]:
df_text_clean = df_text_clean.rename(columns = {0: 'Text', 1: 'Class'}, inplace = False)
df_text_clean.head()

,Text,Class
0,perola negra tais araujo espera batalhadora ro...,tvt
1,cuidados pele negra a pele negra tendencia res...,fam
2,passagem vitoria cara hoje a partir hoje tarif...,cid
3,mauricio prates repasse recurso federal vai ga...,at2
4,paciente coracao novo hospital o primeiro paci...,cid


In [14]:
from sklearn.feature_extraction.text import TfidfTransformer # tf-idf
from sklearn.feature_extraction.text import CountVectorizer # tf

In [15]:
classes = []
for row in df_text_clean.itertuples(index=True, name='Pandas'):  
    if row.Class not in classes:
        classes.append(row.Class)
classes

['tvt', 'fam', 'cid', 'at2', 'mul', 'inf', 'esp', 'mic']

In [16]:
 classes =  ["at2","sro","inf","imo","ept","esp","fam","tvt","cid","mul","pot","tav","mic","opi","poc","reg","eco","bro","cit","con","int"]

## TF

In [17]:
vectorizer = CountVectorizer()
TF = vectorizer.fit_transform(df_text_clean.Text)
features_names =  vectorizer.get_feature_names()

## TF-IDF

In [18]:
from sklearn.pipeline import Pipeline

In [19]:
pipe = Pipeline([('count', CountVectorizer(vocabulary=features_names)),
                 ('tfid', TfidfTransformer())]).fit(df_text_clean.Text)

In [20]:
lista_palavras = []
for row in tqdm(df_text_clean.itertuples(index=True, name='Pandas')):
    palavras = row.Text.split(" ")
    for palavra in palavras:
        if(not len(palavra) <= 2):
            if(palavra not in lista_palavras):
                lista_palavras.append(palavra)

69it [00:02, 31.86it/s] 


# frequencia da palavra por documento

In [21]:
df_relacao_doc = pd.DataFrame()
df_relacao_doc['Index'] = lista_palavras

for i in tqdm(df_text_clean.index):
    doc = df_text_clean.iloc[i].Text
    
    lista_quant =[]
    
    for termo in lista_palavras:
        quant = doc.split(" ").count(termo)
        lista_quant.append(quant)
        
    df_relacao_doc[f'{i}'] = lista_quant
    
    
df_relacao_doc = df_relacao_doc.set_index('Index')
df_relacao_doc.head()

100%|██████████████████████████████████████████████████████████████████████████████████| 69/69 [00:23<00:00,  2.90it/s]


,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,64,65,66,67,68
Index,,,,,,,,,,,,,,,,,,,,,
perola,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
negra,10,7,0,0,0,0,0,0,0,2,...,0,0,0,0,1,0,0,0,0,0
tais,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
araujo,3,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
espera,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# frequencia palavra por classe

In [22]:
dic_doc_class = {}

for classe in tqdm(classes):
    lista_doc = []
    for row in df_text_clean.itertuples(index=True, name='Pandas'):  
        if (str(classe) == row.Class):
            lista_doc.append(row.Text)
    dic_doc_class[f'{classe}'] = lista_doc

100%|████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 2330.72it/s]


In [23]:
df_relacao_class = pd.DataFrame()
df_relacao_class['Index'] = lista_palavras
for key in tqdm(dic_doc_class):
    lista_docs =  dic_doc_class[key]
    docs = ""
    for doc in lista_docs:
        docs += f' {doc}'
        
    doc = docs
    
    lista_quant =[]
    
    for termo in lista_palavras:
        quant = doc.split(" ").count(termo)
        lista_quant.append(quant)
        
    df_relacao_class[f'{key}'] = lista_quant
    
    
df_relacao_class = df_relacao_class.set_index('Index')

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:26<00:00,  1.28s/it]


In [24]:
df_relacao_class.head()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
perola,0,0,0,0,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
negra,3,0,0,0,0,1,7,10,0,0,...,0,0,0,0,0,0,0,0,0,0
tais,0,0,0,0,0,1,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
araujo,0,0,0,0,0,0,0,4,1,0,...,0,0,0,0,0,0,0,0,0,0
espera,4,0,0,0,0,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0


# ni - o numero  de documentos em que um determinado termo ki aparece

In [25]:
dic_ni = {}

for termo in tqdm(lista_palavras):
    quant = 0 
    for row in df_text_clean.itertuples(index=True, name='Pandas'):
        doc = row.Text
        if(termo in doc):
            quant += 1
    dic_ni[f'{termo}'] = quant

100%|████████████████████████████████████████████████████████████████████████████| 9472/9472 [00:05<00:00, 1651.04it/s]


In [26]:
for i in sorted(dic_ni, key = dic_ni.get, reverse=True):
    print(f'key=>{i}, maior=>{dic_ni[]}')
    break

SyntaxError: invalid syntax (<fstring>, line 1)

# a - quantidade de documentos que pertence a uma classe e posuem o termo ki

#  c - o numero  de documentos que pertencem a Cp, mas não possuem o termo ki 

In [27]:
df_a = pd.DataFrame()
df_a['Index'] = lista_palavras

df_c = pd.DataFrame()
df_c['Index'] = lista_palavras

for key in tqdm(dic_doc_class):
    lista_docs =  dic_doc_class[key]
    lista_quant_a =[]
    lista_quant_c =[]

    for termo in lista_palavras:
        quant_a = 0
        quant_c = 0
        
        for doc in lista_docs:
            if(termo in doc):
                quant_a += 1
            else:
                quant_c += 1
                
        lista_quant_a.append(quant_a)
        lista_quant_c.append(quant_c)
        
    df_a[f'{key}'] = lista_quant_a
    df_c[f'{key}'] = lista_quant_c
    
    
df_a = df_a.set_index('Index')
df_c = df_c.set_index('Index')

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:02<00:00,  8.48it/s]


# df_a

In [28]:
df_a.head()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
perola,2,0,0,0,0,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
negra,2,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
tais,1,0,1,0,0,1,0,1,3,0,...,0,0,0,0,0,0,0,0,0,0
araujo,0,0,0,0,0,0,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0
espera,10,0,0,0,0,2,3,2,8,0,...,0,0,0,0,0,0,0,0,0,0


# df_c

In [29]:
df_c.head()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
perola,28,0,3,0,0,4,4,4,19,1,...,0,1,0,0,0,0,0,0,0,0
negra,28,0,3,0,0,3,4,5,19,1,...,0,1,0,0,0,0,0,0,0,0
tais,29,0,2,0,0,3,5,5,16,1,...,0,1,0,0,0,0,0,0,0,0
araujo,30,0,3,0,0,4,5,4,18,1,...,0,1,0,0,0,0,0,0,0,0
espera,20,0,3,0,0,2,2,4,11,1,...,0,1,0,0,0,0,0,0,0,0


# b - a quantidade de documentos que possuem o termo ki, mas não pertence a Cp

In [30]:
df_b = pd.DataFrame()
df_b['Index'] = lista_palavras

df_d = pd.DataFrame()
df_d['Index'] = lista_palavras

for key in tqdm(dic_doc_class):
    lista_quant_b =[]
    lista_quant_d =[]
    
    for termo in lista_palavras:
        quant_b = 0
        quant_d = 0
        for classe in classes:
            if(classe != key):
                lista_docs =  dic_doc_class[classe]
                
                for doc in lista_docs:
                    if(termo in doc):
                        quant_b += 1
                    else:
                        quant_d += 1
                        
        lista_quant_b.append(quant_b)
        lista_quant_d.append(quant_d)
        
    df_b[f'{key}'] = lista_quant_b
    df_d[f'{key}'] = lista_quant_d
    
df_b = df_b.set_index('Index')
df_d = df_d.set_index('Index')

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:42<00:00,  2.01s/it]


# df_b

In [31]:
df_b.head()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
perola,3,5,5,5,5,5,4,3,5,5,...,5,5,5,5,5,5,5,5,5,5
negra,3,5,5,5,5,4,4,4,5,5,...,5,5,5,5,5,5,5,5,5,5
tais,6,7,6,7,7,6,7,6,4,7,...,7,7,7,7,7,7,7,7,7,7
araujo,3,3,3,3,3,3,3,1,2,3,...,3,3,3,3,3,3,3,3,3,3
espera,15,25,25,25,25,23,22,23,17,25,...,25,25,25,25,25,25,25,25,25,25


# df_d

In [32]:
df_d.head()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
perola,36,64,61,64,64,60,60,60,45,63,...,64,63,64,64,64,64,64,64,64,64
negra,36,64,61,64,64,61,60,59,45,63,...,64,63,64,64,64,64,64,64,64,64
tais,33,62,60,62,62,59,57,57,46,61,...,62,61,62,62,62,62,62,62,62,62
araujo,36,66,63,66,66,62,61,62,48,65,...,66,65,66,66,66,66,66,66,66,66
espera,24,44,41,44,44,42,42,40,33,43,...,44,43,44,44,44,44,44,44,44,44


## ICF

In [33]:
import math 

In [34]:
def get_row_class(row, classe):
    dic_classe = {
        'at2':row.at2,
        'sro':row.sro,
        'inf':row.inf,
        'imo':row.imo,
        'ept':row.ept,
        'esp':row.esp,
        'fam':row.fam,
        'tvt':row.tvt,
        'cid':row.cid,
        'mul':row.mul,
        'pot':row.pot,
        'tav':row.tav,
        'mic':row.mic,
        'opi':row.opi,
        'poc':row.poc,
        'reg':row.reg,
        'eco':row.eco,
        'bro':row.bro,
        'cit':row.cit,
        'con':row.con,
        'int':row.int
    }
    return dic_classe[f'{classe}']
    

In [35]:
# conta quantas classes o termo ki aparece
def num_ki_em_categoria():

    dic_num_ki_em_categoria ={}
    for row in tqdm(df_relacao_class.itertuples(index=True, name='Pandas')):
        ci = 0
        for classe in classes:
            if get_row_class(row,classe)!= 0:
                ci += 1
        dic_num_ki_em_categoria[f'{row.Index}'] = ci
        
    return dic_num_ki_em_categoria

# Bag of Word

In [36]:
bag_of_word = {}
bag_of_word['a'] = df_a
bag_of_word['b'] = df_b
bag_of_word['c'] = df_c
bag_of_word['d'] = df_d
bag_of_word['freq_pal_class'] = df_relacao_class
bag_of_word['freq_pal_doc'] = df_relacao_doc
bag_of_word['ci'] = num_ki_em_categoria()
bag_of_word['ni'] = dic_ni

9472it [00:00, 30260.62it/s]


In [37]:
def calculeteICF():
    dic_num_ki_em_categoria = bag_of_word['ci']
    icfs = {}
    C = len(classes)
    for key in dic_num_ki_em_categoria:
        Ci = dic_num_ki_em_categoria[key]
        valor = math.log((1 + (C/Ci)))
        icfs[f'{key}'] = valor
    return icfs

In [61]:
def CalculateRF():
    df_rf = pd.DataFrame()
    df_rf['Index'] = lista_palavras
    for termo in lista_palavras:
        for classe in classes:            
            a =  bag_of_word['a'].loc[termo,classe]
            b =  bag_of_word['b'].loc[termo,classe]
            value =  math.log(2 + (a / (max(1,b))))
            df_rf[f'{classe}'] = value
            
    df_rf =  df_rf.set_index('Index')
    return (df_rf)

In [58]:
def CalculateIDF():
    idfDict = {}
    N = len(df_text_clean)

    for termo, value in tqdm(bag_of_word['ni'].items()):
        idfDict[termo] = math.log10(N / (float(value)))
        
    return(idfDict)

In [78]:
df_rf = CalculateRF()
idfDict = CalculateIDF()

100%|█████████████████████████████████████████████████████████████████████████| 9472/9472 [00:00<00:00, 1183979.96it/s]


In [80]:
def CalculateRF_IDF():
    df_rfIdf = pd.DataFrame()
    df_rfIdf['Index'] = df_rf.index
    
    for index in df_rf.index:
        for classe in classes:            
            valueRF =  df_rf.loc[index,classe]
            valueIdf =  idfDict[index]
            value =  valueRF * valueIdf
            df_rfIdf[f'{classe}'] = value
            
    df_rfIdf =  df_rfIdf.set_index('Index')
    return (df_rfIdf)

In [81]:
CalculateRF_IDF()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
perola,2.020182,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,...,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593
negra,2.020182,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,...,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593
tais,2.020182,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,...,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593
araujo,2.020182,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,...,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593
espera,2.020182,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,...,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vertentes,2.020182,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,...,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593
alternativas,2.020182,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,...,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593
sony/,2.020182,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,...,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593,1.274593


In [39]:
CalculeteRF()

,at2,sro,inf,imo,ept,esp,fam,tvt,cid,mul,...,tav,mic,opi,poc,reg,eco,bro,cit,con,int
Index,,,,,,,,,,,,,,,,,,,,,
perola,1.098612,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147
negra,1.098612,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147
tais,1.098612,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147
araujo,1.098612,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147
espera,1.098612,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vertentes,1.098612,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147
alternativas,1.098612,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147
sony/,1.098612,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147


In [40]:
icfs = calculeteICF() 
icfs

{'perola': 2.4423470353692043,
 'negra': 1.8325814637483102,
 'tais': 2.4423470353692043,
 'araujo': 2.4423470353692043,
 'espera': 2.0794415416798357,
 'batalhadora': 3.091042453358316,
 'romantica': 2.4423470353692043,
 'preta': 3.091042453358316,
 'sirva': 3.091042453358316,
 'exemplo': 1.5040773967762742,
 'jovens': 1.8325814637483102,
 'negros': 2.4423470353692043,
 'gravou': 2.4423470353692043,
 'nome': 1.8325814637483102,
 'historia': 1.8325814637483102,
 'televisiva': 3.091042453358316,
 'pais': 1.5040773967762742,
 '1996': 2.4423470353692043,
 'anos': 1.5040773967762742,
 'viveu': 3.091042453358316,
 'xica': 3.091042453358316,
 'silva': 1.6486586255873816,
 'extinta': 3.091042453358316,
 'manchete': 3.091042453358316,
 'tornou-se': 2.4423470353692043,
 'primeira': 1.5040773967762742,
 'ser': 1.3862943611198906,
 'protagonista': 2.4423470353692043,
 'novela': 2.4423470353692043,
 'brasil': 1.6486586255873816,
 'agora': 1.5040773967762742,
 'atriz': 2.0794415416798357,
 'carioca

In [ ]:
df_icf = pd.DataFrame()
df_icf['Index'] = df_text_clean.index

for termo in tqdm(lista_palavras):
    
    list_icfs = []
    
    for i in df_text_clean.index:
        doc = df_text_clean.iloc[i].Text
        if termo in doc.split(" "):
            list_icfs.append(icfs[termo])
        else:
            list_icfs.append(0)
    
    df_icf[f'{termo}'] = list_icfs
    
df_icf = df_icf.set_index('Index')

In [ ]:
df_icf

In [ ]:
teste = pd.DataFrame.from_dict(icfs, orient='index')

In [ ]:
teste.head()

## Machine Learning

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve,classification_report,r2_score
from sklearn import model_selection as ms
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [ ]:
le = LabelEncoder()

In [ ]:
def toNumeric(nome_coluna, coluna):
    df_text_clean[nome_coluna] = le.fit_transform (coluna)

In [ ]:
toNumeric('numeric_class',df_text_clean.Class)

In [ ]:
# features
X = TF

In [ ]:
#response
Y = df_text_clean['numeric_class'].copy()

In [ ]:
x_train, x_test, y_train, y_test = ms.train_test_split(X,Y,test_size=0.33, random_state=5)

In [ ]:
model_knn = KNeighborsClassifier(n_neighbors=1)

In [ ]:
model_knn.fit(x_train, y_train)

In [ ]:
pred = model_knn.predict_proba(x_test)

In [ ]:
plot_roc_curve(np.array(y_test), pred[:,1])

# Imports do EDA

In [ ]:
# print da curva ROC
def plot_roc_curve(y_true, y_score, figsize=(10,6)):
    fpr, tpr, _ = roc_curve(y_true, y_score, pos_label=2)
    plt.figure(figsize=figsize)
    auc_value = roc_auc_score(y_true, y_score)
    plt.plot(fpr, tpr, color='orange', label='ROC curve (area = %0.2f)' % auc_value)
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('Taxa de falso positivo')
    plt.ylabel('Taxa de verdadeiro positivo')
    plt.title('Curva ROC')
    plt.legend()
    lt.show()

In [ ]:
#precision    recall  f1-score   support
print(classification_report(y_test, pred, labels=[1, 2, 3]))

In [ ]:
# AUC
print('AUC: %0.2f' % roc_auc_score(y_test, pred))

## Número de amostras de cade classe


In [ ]:
import seaborn as sns

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.countplot(x = 'Class',data = textDataDrame)